In [2]:
import os
from google.colab import drive
import numpy as np
import tensorflow.keras as krs
from keras.datasets import mnist
from PIL import Image, ImageFilter
drive.mount('/content/drive')
os.chdir('drive/MyDrive/sukhanich-lapochka/')

Mounted at /content/drive


In [3]:
def imageprepare(argv):
    im = Image.open(argv, mode="r").convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    newImage = Image.new('L', (28, 28), (255))  # creates white canvas of 28x28 pixels

    if width > height:
        nheight = int(round((20.0 / width * height), 0))
        if nheight == 0:
            nheight = 1
        img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((28 - nheight) / 2), 0))
        newImage.paste(img, (4, wtop))
    else:
        nwidth = int(round((20.0 / height * width), 0))
        if nwidth == 0:
            nwidth = 1
        img = im.resize((nwidth, 20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((28 - nwidth) / 2), 0))
        newImage.paste(img, (wleft, 4))

    tv = list(newImage.getdata())

    tva = [(255 - x) * 1.0 / 255.0 for x in tv]
    return tva

In [6]:
# # Load MNIST dataset
# (trainx, trainy), (testx, testy) = mnist.load_data()
# all_image = (trainx/255.0-0.5)*1.999
# all_image = np.expand_dims(all_image, axis=3)

# Define encoder model
nw = nh = 28
num_hide = 98
encoder = krs.models.load_model("encoder_model.h5")
# out_vec = encoder.predict(all_image)

# Define classifier model
num_classes = 10
classificator = krs.models.load_model("classificator_model.h5")

# Load trained weights

# Directory containing the images
image_dir = 'images/'

for i in range(10):
    for filename in os.listdir(f'{image_dir}/{i}'):
        img_path = os.path.join(f'{image_dir}/{i}', filename)
        x = [imageprepare(img_path)]  # This returns a list of length 1
        eps = np.array(x)  # Convert the list to a numpy array
        eps = eps.reshape(-1, 28, 28, 1)  # Reshape to match the input shape expected by the encoder
        out_vec = encoder.predict(eps)  # Predict using the encoder model
        predictions = classificator.predict(out_vec)
        predicted_label = np.argmax(predictions)
        print(f"Image: {img_path}, Predicted Label: {predicted_label}")


<ipython-input-3-4feafff7a1e3>:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = im.resize((nwidth, 20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)


1/1 [==============================] - 1s 717ms/step
Image: images//0/test_image8.jpg, Predicted Label: 8
1/1 [==============================] - 0s 19ms/step
Image: images//0/test_image6.jpg, Predicted Label: 7
1/1 [==============================] - 0s 19ms/step
Image: images//0/test_image4.jpg, Predicted Label: 8
1/1 [==============================] - 0s 33ms/step
Image: images//0/test_image5.jpg, Predicted Label: 7
1/1 [==============================] - 0s 28ms/step
Image: images//0/test_image0.jpg, Predicted Label: 8
1/1 [==============================] - 0s 28ms/step
Image: images//0/test_image2.jpg, Predicted Label: 7
1/1 [==============================] - 0s 34ms/step
Image: images//0/test_image3.jpg, Predicted Label: 1
1/1 [==============================] - 0s 31ms/step
Image: images//0/test_image7.jpg, Predicted Label: 9
1/1 [==============================] - 0s 19ms/step
Image: images//0/test_image1.jpg, Predicted Label: 8
1/1 [==============================] - 0s 21ms/step
Im

<ipython-input-3-4feafff7a1e3>:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)


Image: images//2/test_image0.jpg, Predicted Label: 7
1/1 [==============================] - 0s 20ms/step
Image: images//2/test_image9.jpg, Predicted Label: 1
1/1 [==============================] - 0s 19ms/step
Image: images//2/test_image6.jpg, Predicted Label: 7
1/1 [==============================] - 0s 20ms/step
Image: images//2/test_image1.jpg, Predicted Label: 1
1/1 [==============================] - 0s 19ms/step
Image: images//2/test_image10.jpg, Predicted Label: 7
1/1 [==============================] - 0s 30ms/step
Image: images//2/test_image8.jpg, Predicted Label: 7
1/1 [==============================] - 0s 27ms/step
Image: images//2/test_image3.jpg, Predicted Label: 7
1/1 [==============================] - 0s 27ms/step
Image: images//2/test_image7.jpg, Predicted Label: 7
1/1 [==============================] - 0s 31ms/step
Image: images//3/test_image10.jpg, Predicted Label: 7
1/1 [==============================] - 0s 36ms/step
Image: images//3/test_image1.jpg, Predicted Label: 1
